In [99]:
import pandas as pd
from __future__ import division
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn as sk
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pickle
import datetime
source_folder = 'C:\\Users\\EMBA\\Documents\\Technical_Training\\Machine Learning - Stock Market trading\\Stock\\Data\\Featurized\\'
target_folder = 'C:\\Users\\EMBA\\Documents\\Technical_Training\\Machine Learning - Stock Market trading\\Stock\\Model\\2017-11-25\\'
#end_date = datetime.date.today()
end_date = '2017-11-24'

In [100]:
stocks = ['20MICRONS.NS',
'3IINFOTECH.NS',
'3MINDIA.NS',
'5PAISA.NS',
'63MOONS.NS',
'8KMILES.NS',
'A2ZINFRA.NS',
'AARTIDRUGS.NS',
'AARTIIND.NS',
'AARVEEDEN.NS',
'AARVI.NS',
'ABAN.NS',
'ABB.NS',
'ABBOTINDIA.NS',
'ABCAPITAL.NS',
'ABFRL.NS',
'ABGSHIP.NS',
'ABMINTLTD.NS',
'ACC.NS',
'ACCELYA.NS',
'ACCORD.NS',
'ACE.NS',
'ACEINTEG.NS',
'ADANIENT.NS',
'ADANIPORTS.NS',
'ADANIPOWER.NS',
'ADANITRANS.NS',
'ADFFOODS.NS',
'ADHUNIK.NS',
'ADHUNIKIND.NS',
'ADLABS.NS',
'ADORWELD.NS',
'ADSL.NS',
'ADVANIHOTR.NS',
'ADVENZYMES.NS',
'AEGISCHEM.NS',
'AGARIND.NS',
'AGCNET.NS',
'AGLSL.NS',
'AGRITECH.NS',
'AGROPHOS.NS',
'AHLEAST.NS',
'AHLUCONT.NS',
'AHLWEST.NS',
'AIAENG.NS',
'AIFL.NS',
'AIRAN.NS',
'AIROLAM.NS',
'AISL.NS',
'AJANTPHARM.NS',
'AJMERA.NS',
'AKSHARCHEM.NS',
'AKSHOPTFBR.NS',
'AKZOINDIA.NS',
'ALANKIT.NS',
'ALBERTDAVD.NS',
'ALBK.NS',
'ALCHEM.NS',
'ALEMBICLTD.NS',
'ALICON.NS',
'ALKALI.NS',
'ALKEM.NS',
'ALKYLAMINE.NS',
'ALLCARGO.NS',
'ALLSEC.NS',
'ALMONDZ.NS',
'ALOKTEXT.NS',
'ALPA.NS',
'ALPHAGEO.NS',
'ALPSINDUS.NS',
'AMARAJABAT.NS',
'AMBIKCO.NS',
'AMBUJACEM.NS',
'AMDIND.NS',
'AMRUTANJAN.NS',
'AMTEKAUTO.NS',
'ANANTRAJ.NS',
'ANDHRABANK.NS',
'ANDHRACEMT.NS',
'ANDHRSUGAR.NS',
'ANGIND.NS',
'ANIKINDS.NS',
'ANSALAPI.NS',
'ANSALHSG.NS',
'ANTGRAPHIC.NS',
'APARINDS.NS',
'APCL.NS',
'APCOTEXIND.NS',
'APEX.NS',
'APLAPOLLO.NS',
'APLLTD.NS',
'APOLLOHOSP.NS',
'APOLLOTYRE.NS',
'APOLSINHOT.NS',
'APTECHT.NS',
'ARCHIDPLY.NS',
'ARCHIES.NS',
'ARCOTECH.NS',
'ARIES.NS',
'ARIHANT.NS',
'ARIHANTSUP.NS',
'ARMANFIN.NS',
'AROGRANITE.NS',
'ARROWGREEN.NS',
'ARROWTEX.NS',
'ARSHIYA.NS',
'ARSSINFRA.NS',
'ARVIND.NS',
'ARVSMART.NS',
'ASAHIINDIA.NS',
'ASAHISONG.NS',
'ASAL.NS',
'ASHAPURMIN.NS',
'ASHIANA.NS',
'ASHIMASYN.NS',
'ASHOKA.NS',
'ASHOKLEY.NS',
'ASIANHOTNR.NS',
'ASIANPAINT.NS',
'ASIANTILES.NS',
'ASPINWALL.NS',
'ASSAMCO.NS',
'ASTEC.NS',
'ASTRAL.NS',
'ASTRAMICRO.NS',
'ASTRAZEN.NS',
'ATFL.NS',
'ATLANTA.NS',
'ATLASCYCLE.NS',
'ATUL.NS',
'ATULAUTO.NS',
'AUBANK.NS',
'AURDIS.NS',
'AURIONPRO.NS',
'AUROPHARMA.NS',
'AUSOMENT.NS',
'AUTOAXLES.NS',
'AUTOIND.NS',
'AUTOLITIND.NS',
'AVADHSUGAR.NS',
'AVANTIFEED.NS',
'AVTNPL.NS',
'AXISBANK.NS',
'AXISCADES.NS',
'AXISGOLD.NS',
'AXISNIFTY.NS',
'AYMSYNTEX.NS',
'BAFNAPHARM.NS',
'BAGFILMS.NS',
'BAJAJ-AUTO.NS',
'BAJAJCORP.NS',
'BAJAJELEC.NS',
'BAJAJFINSV.NS',
'BAJAJHIND.NS',
'BAJAJHLDNG.NS',
'BAJFINANCE.NS',
'BALAJITELE.NS',
'BALAMINES.NS',
'BALAXI.NS',
'BALKRISHNA.NS',
'BALKRISIND.NS',
'BALLARPUR.NS',
'BALMLAWRIE.NS',
'BALPHARMA.NS',
'BALRAMCHIN.NS',
'BANARBEADS.NS',
'BANARISUG.NS',
'BANCOINDIA.NS',
'BANG.NS',
'BANKBARODA.NS',
'BANKBEES.NS',
'BANKINDIA.NS',
'BANSAL.NS',
'BANSWRAS.NS',
'BARTRONICS.NS',
'BASF.NS',
'BASML.NS',
'BATAINDIA.NS',
'BAYERCROP.NS',
'BBL.NS',
'BBTC.NS',
'BEARDSELL.NS',
'BEDMUTHA.NS',
'BEL.NS',
'BEML.NS',
'BEPL.NS',
'BERGEPAINT.NS',
'BETA.NS',
'BFINVEST.NS',
'BFUTILITIE.NS',
'BGLOBAL.NS',
'BGRENERGY.NS',
'BHAGERIA.NS',
'BHAGYANGR.NS',
'BHAGYAPROP.NS',
'BHANDARI.NS',
'BHARATFIN.NS',
'BHARATFORG.NS',
'BHARATGEAR.NS',
'BHARATIDIL.NS',
'BHARATRAS.NS',
'BHARATWIRE.NS',
'BHARTIARTL.NS',
'BHEL.NS',
'BHUSANSTL.NS',
'BIGBLOC.NS',
'BIL.NS',
'BILENERGY.NS',
'BILPOWER.NS',
'BINANIIND.NS',
'BINDALAGRO.NS',
'BIOCON.NS',
'BIOFILCHEM.NS',
'BIRLACABLE.NS',
'BIRLACORPN.NS',
'BIRLAMONEY.NS',
'BKMINDST.NS',
'BLBLIMITED.NS',
'BLISSGVS.NS',
'BLKASHYAP.NS',
'BLS.NS',
'BLUEBLENDS.NS',
'BLUECOAST.NS',
'BLUEDART.NS',
'BLUESTARCO.NS',
'BODALCHEM.NS',
'BOHRA.NS',
'BOMDYEING.NS',
'BOSCHLTD.NS',
'BPCL.NS',
'BPL.NS',
'BRFL.NS',
'BRIGADE.NS',
'BRITANNIA.NS',
'BRNL.NS',
'BROOKS.NS',
'BSE.NS',
'BSELINFRA.NS',
'BSL.NS',
'BSLGOLDETF.NS',
'BSLIMITED.NS',
'BSLNIFTY.NS',
'BURNPUR.NS',
'BUTTERFLY.NS',
'BVCL.NS',
'BYKE.NS',
'CADILAHC.NS',
'CADSYS.NS',
'CALSOFT.NS',
'CAMLINFINE.NS',
'CANBK.NS',
'CANDC.NS',
'CANFINHOME.NS',
'CANTABIL.NS',
'CAPACITE.NS',
'CAPF.NS',
'CAPLIPOINT.NS',
'CAPTRUST.NS',
'CARBORUNIV.NS',
'CAREERP.NS',
'CARERATING.NS',
'CASTEXTECH.NS',
'CASTROLIND.NS',
'CCCL.NS',
'CCHHL.NS',
'CCL.NS',
'CDSL.NS',
'CEATLTD.NS',
'CEBBCO.NS',
'CELEBRITY.NS',
'CELESTIAL.NS',
'CENTENKA.NS',
'CENTEXT.NS',
'CENTRALBK.NS',
'CENTUM.NS',
'CENTURYPLY.NS',
'CENTURYTEX.NS',
'CERA.NS',
'CEREBRAINT.NS',
'CESC.NS',
'CGCL.NS',
'CGPOWER.NS',
'CHAMBLFERT.NS',
'CHENNPETRO.NS',
'CHOLAFIN.NS',
'CHROMATIC.NS',
'CIGNITITEC.NS',
'CIMMCO.NS',
'CINELINE.NS',
'CINEVISTA.NS',
'CIPLA.NS',
'CLEDUCATE.NS',
'CLNINDIA.NS',
'CMICABLES.NS',
'CMMIPL.NS',
'CNOVAPETRO.NS',
'COALINDIA.NS',
'COCHINSHIP.NS',
'COFFEEDAY.NS',
'COLPAL.NS',
'COMPINFO.NS',
'COMPUSOFT.NS',
'CONCOR.NS',
'CONSOFINVT.NS',
'CONTROLPR.NS',
'CORALFINAC.NS',
'CORDSCABLE.NS',
'COROMANDEL.NS',
'CORPBANK.NS',
'COSMOFILMS.NS',
'COUNCODOS.NS',
'COX&KINGS.NS',
'CPSEETF.NS',
'CREATIVE.NS',
'CREATIVEYE.NS',
'CREST.NS',
'CRISIL.NS',
'CRMFGETF.NS',
'CROMPTON.NS',
'CTE.NS',
'CUB.NS',
'CUBEXTUB.NS',
'CUMMINSIND.NS',
'CUPID.NS',
'CYBERMEDIA.NS',
'CYBERTECH.NS',
'CYIENT.NS',
'DAAWAT.NS',
'DABUR.NS',
'DALMIABHA.NS',
'DALMIASUG.NS',
'DAMODARIND.NS',
'DATAMATICS.NS',
'DBCORP.NS',
'DBL.NS',
'DBREALTY.NS',
'DCAL.NS',
'DCBBANK.NS',
'DCM.NS',
'DCMSHRIRAM.NS',
'DCW.NS',
'DECCANCE.NS',
'DEEPAKFERT.NS',
'DEEPAKNTR.NS',
'DEEPIND.NS',
'DELTACORP.NS',
'DELTAMAGNT.NS',
'DEN.NS',
'DENABANK.NS',
'DENORA.NS',
'DEVIT.NS',
'DFMFOODS.NS',
'DHAMPURSUG.NS',
'DHANBANK.NS',
'DHANUKA.NS',
'DHARSUGAR.NS',
'DHFL.NS',
'DHUNINV.NS',
'DIAMONDYD.NS',
'DIAPOWER.NS',
'DICIND.NS',
'DIGJAMLTD.NS',
'DISHTV.NS',
'DIVISLAB.NS',
'DIXON.NS',
'DLF.NS',
'DLINKINDIA.NS',
'DMART.NS',
'DOLLAR.NS',
'DOLPHINOFF.NS',
'DONEAR.NS',
'DPABHUSHAN.NS',
'DPL.NS',
'DPSCLTD.NS',
'DPWIRES.NS',
'DQE.NS',
'DREDGECORP.NS',
'DRREDDY.NS',
'DSKULKARNI.NS',
'DSSL.NS',
'DTIL.NS',
'DUCON.NS',
'DWARKESH.NS',
'DYNAMATECH.NS',
'DYNPRO.NS',
'EASTSILK.NS',
'EASUNREYRL.NS',
'ECEIND.NS',
'ECLERX.NS',
'ECLFINANCE.NS',
'EDELWEISS.NS',
'EDL.NS',
'EDUCOMP.NS',
'EHFLNCD.NS',
'EICHERMOT.NS',
'EIDPARRY.NS',
'EIFFL.NS',
'EIHAHOTELS.NS',
'EIHOTEL.NS',
'EIMCOELECO.NS',
'EKC.NS',
'ELAND.NS',
'ELECON.NS',
'ELECTCAST.NS',
'ELECTHERM.NS',
'ELGIEQUIP.NS',
'ELGIRUBCO.NS',
'EMAMIINFRA.NS',
'EMAMILTD.NS',
'EMCO.NS',
'EMKAY.NS',
'EMKAYTOOLS.NS',
'EMMBI.NS',
'ENDURANCE.NS',
'ENERGYDEV.NS',
'ENGINERSIN.NS',
'ENIL.NS',
'EON.NS',
'EQ30.NS',
'EQUITAS.NS',
'ERIS.NS',
'EROSMEDIA.NS',
'ESABINDIA.NS',
'ESCORTS.NS',
'ESL.NS',
'ESSARSHPNG.NS',
'ESSDEE.NS',
'ESSELPACK.NS',
'ESTER.NS',
'EUROCERA.NS',
'EUROMULTI.NS',
'EUROTEXIND.NS',
'EVEREADY.NS',
'EVERESTIND.NS',
'EXCEL.NS',
'EXCELCROP.NS',
'EXCELINDUS.NS',
'EXIDEIND.NS',
'FACT.NS',
'FAIRCHEM.NS',
'FCL.NS',
'FCONSUMER.NS',
'FDC.NS',
'FEDDERELEC.NS',
'FEDERALBNK.NS',
'FEL.NS',
'FELDVR.NS',
'FIEMIND.NS',
'FILATEX.NS',
'FINCABLES.NS',
'FINPIPE.NS',
'FLEXITUFF.NS',
'FLFL.NS',
'FMGOETZE.NS',
'FMNL.NS',
'FOCUS.NS',
'FORGE.NS',
'FORTIS.NS',
'FORTUNEFIN.NS',
'FOSECOIND.NS',
'FOURTHDIM.NS',
'FRETAIL.NS',
'FSL.NS',
'GABRIEL.NS',
'GAEL.NS',
'GAIL.NS',
'GAL.NS',
'GAMMNINFRA.NS',
'GAMMONIND.NS',
'GANDHITUBE.NS',
'GANECOS.NS',
'GANESHHOUC.NS',
'GANGESSECU.NS',
'GARDENSILK.NS',
'GARWALLROP.NS',
'GATI.NS',
'GAYAPROJ.NS',
'GDL.NS',
'GEECEE.NS',
'GEEKAYWIRE.NS',
'GEMINI.NS',
'GENESYS.NS',
'GENUSPAPER.NS',
'GENUSPOWER.NS',
'GEOJITFSL.NS',
'GEPIL.NS',
'GESHIP.NS',
'GET&D.NS',
'GHCL.NS',
'GICHSGFIN.NS',
'GICRE.NS',
'GILLANDERS.NS',
'GILLETTE.NS',
'GINNIFILA.NS',
'GIPCL.NS',
'GIRRESORTS.NS',
'GITANJALI.NS',
'GKWLIMITED.NS',
'GLAXO.NS',
'GLENMARK.NS',
'GLOBAL.NS',
'GLOBALVECT.NS',
'GLOBOFFS.NS',
'GLOBUSSPR.NS',
'GMBREW.NS',
'GMDCLTD.NS',
'GMRINFRA.NS',
'GNA.NS',
'GNFC.NS',
'GOACARBON.NS',
'GOCLCORP.NS',
'GODFRYPHLP.NS',
'GODREJAGRO.NS',
'GODREJCP.NS',
'GODREJIND.NS',
'GODREJPROP.NS',
'GOENKA.NS',
'GOKEX.NS',
'GOKUL.NS',
'GOKULAGRO.NS',
'GOLDBEES.NS',
'GOLDENTOBC.NS',
'GOLDIAM.NS',
'GOLDINFRA.NS',
'GOLDIWIN.NS',
'GOLDSHARE.NS',
'GOLDSTAR.NS',
'GOLDTECH.NS',
'GOODLUCK.NS',
'GPIL.NS',
'GPPL.NS',
'GPTINFRA.NS',
'GRANULES.NS',
'GRAPHITE.NS',
'GRASIM.NS',
'GRAVITA.NS',
'GREAVESCOT.NS',
'GREENLAM.NS',
'GREENPLY.NS',
'GREENPOWER.NS',
'GRINDWELL.NS',
'GROBTEA.NS',
'GRPLTD.NS',
'GRUH.NS',
'GSCLCEMENT.NS',
'GSFC.NS',
'GSKCONS.NS',
'GSPL.NS',
'GSS.NS',
'GTL.NS',
'GTLINFRA.NS',
'GTNIND.NS',
'GTNTEX.NS',
'GTPL.NS',
'GUFICBIO.NS',
'GUJALKALI.NS',
'GUJAPOLLO.NS',
'GUJFLUORO.NS',
'GUJGASLTD.NS',
'GUJNRECOKE.NS',
'GUJNREDVR.NS',
'GULFOILLUB.NS',
'GULFPETRO.NS',
'GULPOLY.NS',
'GVKPIL.NS',
'HANUNG.NS',
'HARITASEAT.NS',
'HARRMALAYA.NS',
'HATHWAY.NS',
'HATSUN.NS',
'HAVELLS.NS',
'HBLPOWER.NS',
'HBSTOCK.NS',
'HCC.NS',
'HCG.NS',
'HCL-INSYS.NS',
'HCLTECH.NS',
'HDFC.NS',
'HDFCBANK.NS',
'HDFCLIFE.NS',
'HDFCMFGETF.NS',
'HDFCNIFETF.NS',
'HDFCSENETF.NS',
'HDIL.NS',
'HECPROJECT.NS',
'HEG.NS',
'HEIDELBERG.NS',
'HERCULES.NS',
'HERITGFOOD.NS',
'HEROMOTOCO.NS',
'HESTERBIO.NS',
'HEXATRADEX.NS',
'HEXAWARE.NS',
'HFCL.NS',
'HGS.NS',
'HIGHGROUND.NS',
'HIKAL.NS',
'HIL.NS',
'HILTON.NS',
'HIMATSEIDE.NS',
'HINDALCO.NS',
'HINDCOMPOS.NS',
'HINDCOPPER.NS',
'HINDDORROL.NS',
'HINDMOTORS.NS',
'HINDNATGLS.NS',
'HINDOILEXP.NS',
'HINDPETRO.NS',
'HINDUJAVEN.NS',
'HINDUNILVR.NS',
'HINDZINC.NS',
'HIRECT.NS',
'HISARMETAL.NS',
'HITECH.NS',
'HITECHCORP.NS',
'HITECHGEAR.NS',
'HMT.NS',
'HMVL.NS',
'HONAUT.NS',
'HONDAPOWER.NS',
'HOTELEELA.NS',
'HOVS.NS',
'HPL.NS',
'HSCL.NS',
'HSIL.NS',
'HTMEDIA.NS',
'HUBTOWN.NS',
'HUDCO.NS',
'IBREALEST.NS',
'IBULHSGFIN.NS',
'IBVENTURES.NS',
'ICICIBANK.NS',
'ICICIGI.NS',
'ICICIPRULI.NS',
'ICIL.NS',
'ICRA.NS',
'ICSA.NS',
'IDBI.NS',
'IDBIGOLD.NS',
'IDEA.NS',
'IDFC.NS',
'IDFCBANK.NS',
'IDFNIFTYET.NS',
'IEX.NS',
'IFBAGRO.NS',
'IFBIND.NS',
'IFCI.NS',
'IFGLEXPOR.NS',
'IGARASHI.NS',
'IGL.NS',
'IGPL.NS',
'IIFCL.NS',
'IIFL.NS',
'IIFLFIN.NS',
'IIHFL.NS',
'IITL.NS',
'IL&FSENGG.NS',
'IL&FSTRANS.NS',
'IMFA.NS',
'IMPAL.NS',
'INDBANK.NS',
'INDHOTEL.NS',
'INDIACEM.NS',
'INDIAGLYCO.NS',
'INDIANB.NS',
'INDIANCARD.NS',
'INDIANHUME.NS',
'INDIGO.NS',
'INDIGRID.NS',
'INDLMETER.NS',
'INDNIPPON.NS',
'INDOCO.NS',
'INDORAMA.NS',
'INDOSOLAR.NS',
'INDOTECH.NS',
'INDOTHAI.NS',
'INDOWIND.NS',
'INDRAMEDCO.NS',
'INDSWFTLAB.NS',
'INDSWFTLTD.NS',
'INDTERRAIN.NS',
'INDUSINDBK.NS',
'INEOSSTYRO.NS',
'INFIBEAM.NS',
'INFINITE.NS',
'INFOBEAN.NS',
'INFRABEES.NS',
'INFRATEL.NS',
'INFY.NS',
'INGERRAND.NS',
'INNOVATIVE.NS',
'INOXLEISUR.NS',
'INOXWIND.NS',
'INSECTICID.NS',
'INTEGRA.NS',
'INTELLECT.NS',
'INTENTECH.NS',
'INVENTURE.NS',
'IOB.NS',
'IOC.NS',
'IOLCP.NS',
'IPAPPM.NS',
'IPCALAB.NS',
'IRB.NS',
'IRBINVIT.NS',
'IREDA.NS',
'IRFC.NS',
'ISFT.NS',
'ISMTLTD.NS',
'ITC.NS',
'ITDC.NS',
'ITDCEM.NS',
'ITI.NS',
'IVC.NS',
'IVP.NS',
'IVRCLINFRA.NS',
'IVZINGOLD.NS',
'IZMO.NS',
'J&KBANK.NS',
'JAGRAN.NS',
'JAGSNPHARM.NS',
'JAIBALAJI.NS',
'JAICORPLTD.NS',
'JAIHINDPRO.NS',
'JAINSTUDIO.NS',
'JALAN.NS',
'JAMNAAUTO.NS',
'JASH.NS',
'JAYAGROGN.NS',
'JAYBARMARU.NS',
'JAYNECOIND.NS',
'JAYSREETEA.NS',
'JBCHEPHARM.NS',
'JBFIND.NS',
'JBMA.NS',
'JCHAC.NS',
'JENSONICOL.NS',
'JETAIRWAYS.NS',
'JETKNIT.NS',
'JHS.NS',
'JINDALPHOT.NS',
'JINDALPOLY.NS',
'JINDALSAW.NS',
'JINDALSTEL.NS',
'JINDCOT.NS',
'JINDRILL.NS',
'JINDWORLD.NS',
'JISLDVREQS.NS',
'JISLJALEQS.NS',
'JITFINFRA.NS',
'JKCEMENT.NS',
'JKIL.NS',
'JKLAKSHMI.NS',
'JKPAPER.NS',
'JKTYRE.NS',
'JMA.NS',
'JMCPROJECT.NS',
'JMFINANCIL.NS',
'JMTAUTOLTD.NS',
'JOCIL.NS',
'JPASSOCIAT.NS',
'JPINFRATEC.NS',
'JPOLYINVST.NS',
'JPPOWER.NS',
'JSL.NS',
'JSLHISAR.NS',
'JSWENERGY.NS',
'JSWHL.NS',
'JSWSTEEL.NS',
'JUBILANT.NS',
'JUBLFOOD.NS',
'JUBLINDS.NS',
'JUNIORBEES.NS',
'JUSTDIAL.NS',
'JVLAGRO.NS',
'JYOTHYLAB.NS',
'JYOTISTRUC.NS',
'KABRAEXTRU.NS',
'KAJARIACER.NS',
'KAKATCEM.NS',
'KALPATPOWR.NS',
'KALYANIFRG.NS',
'KAMATHOTEL.NS',
'KAMDHENU.NS',
'KANANIIND.NS',
'KANORICHEM.NS',
'KANSAINER.NS',
'KARMAENG.NS',
'KARURVYSYA.NS',
'KAVVERITEL.NS',
'KAYA.NS',
'KCP.NS',
'KCPSUGIND.NS',
'KDDL.NS',
'KEC.NS',
'KECL.NS',
'KEERTI.NS',
'KEI.NS',
'KELLTONTEC.NS',
'KERNEX.NS',
'KESARENT.NS',
'KESORAMIND.NS',
'KEYCORPSER.NS',
'KGL.NS',
'KHADIM.NS',
'KHAITANLTD.NS',
'KHANDSE.NS',
'KICL.NS',
'KILITCH.NS',
'KINGFA.NS',
'KIOCL.NS',
'KIRIINDUS.NS',
'KIRLOSBROS.NS',
'KIRLOSENG.NS',
'KIRLOSIND.NS',
'KITEX.NS',
'KKCL.NS',
'KMSUGAR.NS',
'KNRCON.NS',
'KOHINOOR.NS',
'KOKUYOCMLN.NS',
'KOLTEPATIL.NS',
'KOPRAN.NS',
'KOTAKBANK.NS',
'KOTAKBKETF.NS',
'KOTAKGOLD.NS',
'KOTAKNIFTY.NS',
'KOTAKNV20.NS',
'KOTAKPSUBK.NS',
'KOTARISUG.NS',
'KOTHARIPET.NS',
'KOTHARIPRO.NS',
'KPIT.NS',
'KPRMILL.NS',
'KRBL.NS',
'KREBSBIO.NS',
'KRIDHANINF.NS',
'KRISHANA.NS',
'KSBPUMPS.NS',
'KSCL.NS',
'KSERASERA.NS',
'KSK.NS',
'KSL.NS',
'KTIL.NS',
'KTKBANK.NS',
'KWALITY.NS',
'L&TFH.NS',
'L&TFINANCE.NS',
'L&TINFRA.NS',
'LAKPRE.NS',
'LAKSHMIEFL.NS',
'LAKSHVILAS.NS',
'LALPATHLAB.NS',
'LAMBODHARA.NS',
'LAOPALA.NS',
'LASA.NS',
'LAURUSLABS.NS',
'LAXMIMACH.NS',
'LCCINFOTEC.NS',
'LEEL.NS',
'LEXUS.NS',
'LFIC.NS',
'LGBBROSLTD.NS',
'LIBAS.NS',
'LIBERTSHOE.NS',
'LICHSGFIN.NS',
'LICNETFGSC.NS',
'LICNETFN50.NS',
'LICNFNHGP.NS',
'LINCOLN.NS',
'LINCPEN.NS',
'LINDEINDIA.NS',
'LIQUIDBEES.NS',
'LITL.NS',
'LML.NS',
'LOKESHMACH.NS',
'LOTUSEYE.NS',
'LOVABLE.NS',
'LOWVOLIWIN.NS',
'LPDC.NS',
'LSIL.NS',
'LT.NS',
'LTI.NS',
'LTTS.NS',
'LUMAXIND.NS',
'LUMAXTECH.NS',
'LUPIN.NS',
'LUXIND.NS',
'LYCOS.NS',
'LYKALABS.NS',
'LYPSAGEMS.NS',
'M&M.NS',
'M&MFIN.NS',
'M100.NS',
'M15RD.NS',
'M50.NS',
'MAANALU.NS',
'MADHAV.NS',
'MADHUCON.NS',
'MADRASFERT.NS',
'MAGADSUGAR.NS',
'MAGMA.NS',
'MAGNUM.NS',
'MAHABANK.NS',
'MAHASTEEL.NS',
'MAHESHWARI.NS',
'MAHINDCIE.NS',
'MAHLIFE.NS',
'MAHLOG.NS',
'MAHSCOOTER.NS',
'MAHSEAMLES.NS',
'MAITHANALL.NS',
'MAJESCO.NS',
'MALUPAPER.NS',
'MANAKALUCO.NS',
'MANAKCOAT.NS',
'MANAKSIA.NS',
'MANAKSTEEL.NS',
'MANALIPETC.NS',
'MANAPPURAM.NS',
'MANAV.NS',
'MANDHANA.NS',
'MANGALAM.NS',
'MANGCHEFER.NS',
'MANGLMCEM.NS',
'MANGTIMBER.NS',
'MANINDS.NS',
'MANINFRA.NS',
'MANPASAND.NS',
'MANUGRAPH.NS',
'MARALOVER.NS',
'MARATHON.NS',
'MARICO.NS',
'MARKSANS.NS',
'MARUTI.NS',
'MASFIN.NS',
'MASTEK.NS',
'MATRIMONY.NS',
'MAWANASUG.NS',
'MAXINDIA.NS',
'MAXVIL.NS',
'MAYURUNIQ.NS',
'MAZDA.NS',
'MBAPL.NS',
'MBECL.NS',
'MBLINFRA.NS',
'MCDHOLDING.NS',
'MCDOWELL-N.NS',
'MCLEODRUSS.NS',
'MCX.NS',
'MEGASOFT.NS',
'MEGH.NS',
'MENONBE.NS',
'MEP.NS',
'MERCATOR.NS',
'MERCK.NS',
'METALFORGE.NS',
'METKORE.NS',
'MFSL.NS',
'MGL.NS',
'MHRIL.NS',
'MIC.NS',
'MIDCAPIWIN.NS',
'MILTON.NS',
'MINDACORP.NS',
'MINDAIND.NS',
'MINDTECK.NS',
'MINDTREE.NS',
'MIRCELECTR.NS',
'MIRZAINT.NS',
'MKPL.NS',
'MMFL.NS',
'MMTC.NS',
'MOHITIND.NS',
'MOHOTAMILL.NS',
'MOIL.NS',
'MOLDTECH.NS',
'MOLDTKPAC.NS',
'MOMAI.NS',
'MONNETISPA.NS',
'MONSANTO.NS',
'MONTECARLO.NS',
'MORARJEE.NS',
'MOREPENLAB.NS',
'MOSERBAER.NS',
'MOTHERSUMI.NS',
'MOTILALOFS.NS',
'MOTOGENFIN.NS',
'MPHASIS.NS',
'MPSLTD.NS',
'MRF.NS',
'MRPL.NS',
'MSPL.NS',
'MTEDUCARE.NS',
'MTNL.NS',
'MUKANDENGG.NS',
'MUKANDLTD.NS',
'MUKTAARTS.NS',
'MUNJALAU.NS',
'MUNJALSHOW.NS',
'MURUDCERA.NS',
'MUTHOOTCAP.NS',
'MUTHOOTFIN.NS',
'N100.NS',
'NABARD.NS',
'NACLIND.NS',
'NAGAFERT.NS',
'NAGAROIL.NS',
'NAGREEKCAP.NS',
'NAGREEKEXP.NS',
'NAHARCAP.NS',
'NAHARINDUS.NS',
'NAHARPOLY.NS',
'NAHARSPING.NS',
'NAKODA.NS',
'NANDANI.NS',
'NATCOPHARM.NS',
'NATHBIOGEN.NS',
'NATIONALUM.NS',
'NATNLSTEEL.NS',
'NAUKRI.NS',
'NAVINFLUOR.NS',
'NAVKARCORP.NS',
'NAVNETEDUL.NS',
'NBCC.NS',
'NBIFIN.NS',
'NBVENTURES.NS',
'NCC.NS',
'NCLIND.NS',
'NDGL.NS',
'NDL.NS',
'NDTV.NS',
'NECCLTD.NS',
'NECLIFE.NS',
'NELCAST.NS',
'NELCO.NS',
'NESCO.NS',
'NESTLEIND.NS',
'NETWORK18.NS',
'NEULANDLAB.NS',
'NEXTMEDIA.NS',
'NFL.NS',
'NH.NS',
'NHAI.NS',
'NHBTF2014.NS',
'NHBTF2023.NS',
'NHPC.NS',
'NIACL.NS',
'NIBL.NS',
'NIF100IWIN.NS',
'NIFTYBEES.NS',
'NIFTYEES.NS',
'NIFTYIWIN.NS',
'NIITLTD.NS',
'NIITTECH.NS',
'NILAINFRA.NS',
'NILKAMAL.NS',
'NIPPOBATRY.NS',
'NITCO.NS',
'NITESHEST.NS',
'NITINFIRE.NS',
'NITINSPIN.NS',
'NKIND.NS',
'NLCINDIA.NS',
'NMDC.NS',
'NOCIL.NS',
'NOIDATOLL.NS',
'NRAIL.NS',
'NRBBEARING.NS',
'NSIL.NS',
'NTL.NS',
'NTPC.NS',
'NUCLEUS.NS',
'NUTEK.NS',
'NV20IWIN.NS',
'OBEROIRLTY.NS',
'OCCL.NS',
'OCL.NS',
'OFSS.NS',
'OIL.NS',
'OILCOUNTUB.NS',
'OISL.NS',
'OMAXAUTO.NS',
'OMAXE.NS',
'OMKARCHEM.NS',
'OMMETALS.NS',
'ONELIFECAP.NS',
'ONGC.NS',
'ONMOBILE.NS',
'ONWARDTEC.NS',
'OPTIEMUS.NS',
'OPTOCIRCUI.NS',
'ORBTEXP.NS',
'ORCHIDPHAR.NS',
'ORICONENT.NS',
'ORIENTABRA.NS',
'ORIENTALTL.NS',
'ORIENTBANK.NS',
'ORIENTBELL.NS',
'ORIENTCEM.NS',
'ORIENTHOT.NS',
'ORIENTLTD.NS',
'ORIENTPPR.NS',
'ORIENTREF.NS',
'ORISSAMINE.NS',
'ORTEL.NS',
'ORTINLABSS.NS',
'PAEL.NS',
'PAGEIND.NS',
'PALASHSECU.NS',
'PALREDTEC.NS',
'PANACEABIO.NS',
'PANAMAPET.NS',
'PANORAMUNI.NS',
'PAPERPROD.NS',
'PARABDRUGS.NS',
'PARACABLES.NS',
'PARAGMILK.NS',
'PARSVNATH.NS',
'PASHUPATI.NS',
'PATELENG.NS',
'PATINTLOG.NS',
'PATSPINLTD.NS',
'PBAINFRA.NS',
'PCJEWELLER.NS',
'PDMJEPAPER.NS',
'PDPL.NS',
'PDSMFL.NS',
'PDUMJEIND.NS',
'PDUMJEPULP.NS',
'PEARLPOLY.NS',
'PEL.NS',
'PENIND.NS',
'PENINLAND.NS',
'PENPEBS.NS',
'PERFECT.NS',
'PERSISTENT.NS',
'PETRONENGG.NS',
'PETRONET.NS',
'PFC.NS',
'PFIZER.NS',
'PFOCUS.NS',
'PFS.NS',
'PGEL.NS',
'PGHH.NS',
'PGIL.NS',
'PHILIPCARB.NS',
'PHOENIXLTD.NS',
'PIDILITIND.NS',
'PIIND.NS',
'PILANIINVS.NS',
'PILITA.NS',
'PINCON.NS',
'PIONDIST.NS',
'PIONEEREMB.NS',
'PITTILAM.NS',
'PKTEA.NS',
'PLASTIBLEN.NS',
'PNB.NS',
'PNBGILTS.NS',
'PNBHOUSING.NS',
'PNC.NS',
'PNCINFRA.NS',
'POCHIRAJU.NS',
'PODDARHOUS.NS',
'PODDARMENT.NS',
'POKARNA.NS',
'POLARIS.NS',
'POLYMED.NS',
'POLYPLEX.NS',
'PONNIERODE.NS',
'POWERGRID.NS',
'POWERMECH.NS',
'PPAP.NS',
'PRABHAT.NS',
'PRADIP.NS',
'PRAENG.NS',
'PRAJIND.NS',
'PRAKASH.NS',
'PRAKASHCON.NS',
'PRATIBHA.NS',
'PRECAM.NS',
'PRECOT.NS',
'PRECWIRE.NS',
'PREMEXPLN.NS',
'PREMIER.NS',
'PREMIERPOL.NS',
'PRESSMN.NS',
'PRESTIGE.NS',
'PRICOLLTD.NS',
'PRIMESECU.NS',
'PRISMCEM.NS',
'PROVOGE.NS',
'PROZONINTU.NS',
'PSB.NS',
'PSL.NS',
'PSPPROJECT.NS',
'PSUBNKBEES.NS',
'PTC.NS',
'PTL.NS',
'PULZ.NS',
'PUNJABCHEM.NS',
'PUNJLLOYD.NS',
'PURVA.NS',
'PUSHPREALM.NS',
'PVP.NS',
'PVR.NS',
'QGOLDHALF.NS',
'QUESS.NS',
'QUICKHEAL.NS',
'QUINTEGRA.NS',
'RADAAN.NS',
'RADICO.NS',
'RADIOCITY.NS',
'RAIN.NS',
'RAINBOWPAP.NS',
'RAJESHEXPO.NS',
'RAJOIL.NS',
'RAJRAYON.NS',
'RAJSREESUG.NS',
'RAJTV.NS',
'RALLIS.NS',
'RAMANEWS.NS',
'RAMASTEEL.NS',
'RAMCOCEM.NS',
'RAMCOIND.NS',
'RAMCOSYS.NS',
'RAMKY.NS',
'RAMSARUP.NS',
'RANASUG.NS',
'RANEENGINE.NS',
'RANEHOLDIN.NS',
'RATNAMANI.NS',
'RAYMOND.NS',
'RBL.NS',
'RBLBANK.NS',
'RCF.NS',
'RCOM.NS',
'RECLTD.NS',
'REDINGTON.NS',
'REFEX.NS',
'REGENCERAM.NS',
'RELAXO.NS',
'RELCAPITAL.NS',
'RELCNX100.NS',
'RELCONS.NS',
'RELDIVOPP.NS',
'RELIABLE.NS',
'RELIANCE.NS',
'RELIFIN.NS',
'RELIGARE.NS',
'RELINFRA.NS',
'RELNV20.NS',
'REMSONSIND.NS',
'RENUKA.NS',
'REPCOHOME.NS',
'REPRO.NS',
'RESPONIND.NS',
'REVATHI.NS',
'RHFL.NS',
'RICOAUTO.NS',
'RIIL.NS',
'RJL.NS',
'RKDL.NS',
'RKEC.NS',
'RKFORGE.NS',
'RMCL.NS',
'RMDRIP.NS',
'RML.NS',
'RNAM.NS',
'RNAVAL.NS',
'ROHLTD.NS',
'ROLLT.NS',
'ROLTA.NS',
'ROSSELLIND.NS',
'RPGLIFE.NS',
'RPOWER.NS',
'RPPINFRA.NS',
'RRSLGETF.NS',
'RSSOFTWARE.NS',
'RSWM.NS',
'RSYSTEMS.NS',
'RTNINFRA.NS',
'RTNPOWER.NS',
'RUBYMILLS.NS',
'RUCHINFRA.NS',
'RUCHIRA.NS',
'RUCHISOYA.NS',
'RUPA.NS',
'RUSHIL.NS',
'SABEVENTS.NS',
'SABTN.NS',
'SADBHAV.NS',
'SADBHIN.NS',
'SAGARDEEP.NS',
'SAGCEM.NS',
'SAIL.NS',
'SAKHTISUG.NS',
'SAKSOFT.NS',
'SAKUMA.NS',
'SALASAR.NS',
'SALONA.NS',
'SALORAINTL.NS',
'SALSTEEL.NS',
'SALZERELEC.NS',
'SAMBHAAV.NS',
'SAMTEL.NS',
'SANCO.NS',
'SANDESH.NS',
'SANGAMIND.NS',
'SANGHIIND.NS',
'SANGHVIFOR.NS',
'SANGHVIMOV.NS',
'SANGINITA.NS',
'SANOFI.NS',
'SANWARIA.NS',
'SARDAEN.NS',
'SAREGAMA.NS',
'SARLAPOLY.NS',
'SASKEN.NS',
'SASTASUNDR.NS',
'SATHAISPAT.NS',
'SATIN.NS',
'SBILIFE.NS',
'SBIN.NS',
'SCAPDVR.NS',
'SCHAEFFLER.NS',
'SCHAND.NS',
'SCHNEIDER.NS',
'SCI.NS',
'SDBL.NS',
'SEAMECLTD.NS',
'SECURCRED.NS',
'SEINV.NS',
'SELAN.NS',
'SELMCL.NS',
'SENSEXIWIN.NS',
'SEPOWER.NS',
'SEQUENT.NS',
'SERVALL.NS',
'SERVOTECH.NS',
'SESHAPAPER.NS',
'SETCO.NS',
'SETF10GILT.NS',
'SETFGOLD.NS',
'SETFNIF50.NS',
'SETFNIFBK.NS',
'SETFNN50.NS',
'SEZAL.NS',
'SFL.NS',
'SGBAUG24.NS',
'SGBFEB24.NS',
'SGBJUL25.NS',
'SGBMAR25.NS',
'SGBMAY25.NS',
'SGBNOV23.NS',
'SGBNOV24.NS',
'SGBNOV25.NS',
'SGBOCT25.NS',
'SGBOCT25IV.NS',
'SGBOCT25V.NS',
'SGBSEP24.NS',
'SGL.NS',
'SHAHALLOYS.NS',
'SHAKTIPUMP.NS',
'SHALPAINTS.NS',
'SHANKARA.NS',
'SHANTI.NS',
'SHANTIGEAR.NS',
'SHARDACROP.NS',
'SHARDAMOTR.NS',
'SHARIABEES.NS',
'SHARONBIO.NS',
'SHEMAROO.NS',
'SHILPAMED.NS',
'SHILPI.NS',
'SHIRPUR-G.NS',
'SHIVAMAUTO.NS',
'SHK.NS',
'SHOPERSTOP.NS',
'SHREECEM.NS',
'SHREEPUSHK.NS',
'SHREERAMA.NS',
'SHRENIK.NS',
'SHREYANIND.NS',
'SHREYAS.NS',
'SHRIPISTON.NS',
'SHRIRAMCIT.NS',
'SHRIRAMEPC.NS',
'SHYAMCENT.NS',
'SHYAMTEL.NS',
'SICAGEN.NS',
'SICAL.NS',
'SIEMENS.NS',
'SIGNET.NS',
'SIIL.NS',
'SIL.NS',
'SILINV.NS',
'SIMBHALS.NS',
'SIMPLEX.NS',
'SIMPLEXINF.NS',
'SINTEX.NS',
'SIS.NS',
'SITASHREE.NS',
'SITINET.NS',
'SIYSIL.NS',
'SJVN.NS',
'SKFINDIA.NS',
'SKIL.NS',
'SKIPPER.NS',
'SKMEGGPROD.NS',
'SMARTLINK.NS',
'SMCSRVIRD.NS',
'SMCSRVIRG.NS',
'SMLISUZU.NS',
'SMPL.NS',
'SMSLIFE.NS',
'SMSPHARMA.NS',
'SNOWMAN.NS',
'SOBHA.NS',
'SOLARINDS.NS',
'SOMANYCERA.NS',
'SOMATEX.NS',
'SOMICONVEY.NS',
'SONASTEER.NS',
'SONATSOFTW.NS',
'SORILHOLD.NS',
'SORILINFRA.NS',
'SOTL.NS',
'SOUTHBANK.NS',
'SPAL.NS',
'SPARC.NS',
'SPECIALITY.NS',
'SPENTEX.NS',
'SPHEREGSL.NS',
'SPIC.NS',
'SPICEMOBI.NS',
'SPLIL.NS',
'SPMLINFRA.NS',
'SPTL.NS',
'SPYL.NS',
'SQSBFSI.NS',
'SREEL.NS',
'SREIBNPNCD.NS',
'SREINFRA.NS',
'SRF.NS',
'SRHHYPOLTD.NS',
'SRIPIPES.NS',
'SRIRAM.NS',
'SRSLTD.NS',
'SRTRANSFIN.NS',
'SSWL.NS',
'STAMPEDE.NS',
'STAN.NS',
'STAR.NS',
'STARCEMENT.NS',
'STARPAPER.NS',
'STCINDIA.NS',
'STEELCITY.NS',
'STEELXIND.NS',
'STEL.NS',
'STERLINBIO.NS',
'STERTOOLS.NS',
'STINDIA.NS',
'STRTECH.NS',
'SUBEX.NS',
'SUBROS.NS',
'SUDARSCHEM.NS',
'SUJANAUNI.NS',
'SUMEETINDS.NS',
'SUMMITSEC.NS',
'SUNCLAYLTD.NS',
'SUNDARAM.NS',
'SUNDARMFIN.NS',
'SUNDRMBRAK.NS',
'SUNDRMFAST.NS',
'SUNFLAG.NS',
'SUNILHITEC.NS',
'SUNPHARMA.NS',
'SUNTECK.NS',
'SUNTV.NS',
'SUPERHOUSE.NS',
'SUPERSPIN.NS',
'SUPPETRO.NS',
'SUPRAJIT.NS',
'SUPREMEIND.NS',
'SUPREMEINF.NS',
'SUPREMETEX.NS',
'SURANAIND.NS',
'SURANASOL.NS',
'SURANAT&P.NS',
'SUREVIN.NS',
'SURYALAXMI.NS',
'SURYAROSNI.NS',
'SUTLEJTEX.NS',
'SUVEN.NS',
'SUZLON.NS',
'SWANENERGY.NS',
'SWARAJENG.NS',
'SWELECTES.NS',
'SYMPHONY.NS',
'SYNCOM.NS',
'SYNDIBANK.NS',
'SYNGENE.NS',
'TAINWALCHM.NS',
'TAJGVK.NS',
'TAKE.NS',
'TALBROAUTO.NS',
'TALWALKARS.NS',
'TANLA.NS',
'TANTIACONS.NS',
'TARAJEWELS.NS',
'TARAPUR.NS',
'TARMAT.NS',
'TASTYBITE.NS',
'TATACHEM.NS',
'TATACOFFEE.NS',
'TATACOMM.NS',
'TATAELXSI.NS',
'TATAGLOBAL.NS',
'TATAINVEST.NS',
'TATAMETALI.NS',
'TATAMOTORS.NS',
'TATAMTRDVR.NS',
'TATAPOWER.NS',
'TATASPONGE.NS',
'TATASTEEL.NS',
'TBZ.NS',
'TCFSL.NS',
'TCI.NS',
'TCIDEVELOP.NS',
'TCIEXP.NS',
'TCIFINANCE.NS',
'TCPLPACK.NS',
'TCS.NS',
'TDPOWERSYS.NS',
'TEAMLEASE.NS',
'TECHIN.NS',
'TECHM.NS',
'TECHNO.NS',
'TECHNOFAB.NS',
'TEJASNET.NS',
'TERASOFT.NS',
'TEXINFRA.NS',
'TEXMOPIPES.NS',
'TEXRAIL.NS',
'TFCILTD.NS',
'TFL.NS',
'TGBHOTELS.NS',
'THANGAMAYL.NS',
'THEJO.NS',
'THEMISMED.NS',
'THERMAX.NS',
'THIRUSUGAR.NS',
'THOMASCOOK.NS',
'THOMASCOTT.NS',
'THYROCARE.NS',
'TI.NS',
'TIDEWATER.NS',
'TIFIN.NS',
'TIIL.NS',
'TIINDIA.NS',
'TIJARIA.NS',
'TIL.NS',
'TIMESGTY.NS',
'TIMETECHNO.NS',
'TIMKEN.NS',
'TINPLATE.NS',
'TIPSINDLTD.NS',
'TIRUMALCHM.NS',
'TIRUPATI.NS',
'TITAN.NS',
'TMRVL.NS',
'TNPETRO.NS',
'TNPL.NS',
'TOKYOPLAST.NS',
'TORNTPHARM.NS',
'TORNTPOWER.NS',
'TOTAL.NS',
'TPLPLASTEH.NS',
'TRANSWIND.NS',
'TREEHOUSE.NS',
'TRENT.NS',
'TRF.NS',
'TRIDENT.NS',
'TRIGYN.NS',
'TRIL.NS',
'TRITURBINE.NS',
'TRIVENI.NS',
'TTKHLTCARE.NS',
'TTKPRESTIG.NS',
'TTL.NS',
'TTML.NS',
'TULSI.NS',
'TV18BRDCST.NS',
'TVSELECT.NS',
'TVSMOTOR.NS',
'TVSSRICHAK.NS',
'TVTODAY.NS',
'TVVISION.NS',
'TWL.NS',
'UBL.NS',
'UCALFUEL.NS',
'UCOBANK.NS',
'UFLEX.NS',
'UFO.NS',
'UGARSUGAR.NS',
'UJAAS.NS',
'UJJIVAN.NS',
'ULTRACEMCO.NS',
'UMANGDAIRY.NS',
'UMESLTD.NS',
'UNICHEMLAB.NS',
'UNIENTER.NS',
'UNIONBANK.NS',
'UNIPLY.NS',
'UNITECH.NS',
'UNITEDBNK.NS',
'UNITEDPOLY.NS',
'UNITEDTEA.NS',
'UNITY.NS',
'UNIVCABLES.NS',
'UPL.NS',
'URJA.NS',
'USHAMART.NS',
'USHERAGRO.NS',
'UTINEXT50.NS',
'UTINIFTETF.NS',
'UTISENSETF.NS',
'UTTAMSTL.NS',
'UTTAMSUGAR.NS',
'UVSL.NS',
'V2RETAIL.NS',
'VADILALIND.NS',
'VAIBHAVGBL.NS',
'VAISHALI.NS',
'VAKRANGEE.NS',
'VALUEIND.NS',
'VARDHACRLC.NS',
'VARDMNPOLY.NS',
'VASCONEQ.NS',
'VASWANI.NS',
'VBL.NS',
'VEDL.NS',
'VENKEYS.NS',
'VENUSREM.NS',
'VERTOZ.NS',
'VESUVIUS.NS',
'VETO.NS',
'VGUARD.NS',
'VHL.NS',
'VICEROY.NS',
'VIDEOIND.NS',
'VIDHIING.NS',
'VIJAYABANK.NS',
'VIJIFIN.NS',
'VIJSHAN.NS',
'VIKASECO.NS',
'VIMTALABS.NS',
'VINATIORGA.NS',
'VINDHYATEL.NS',
'VINYLINDIA.NS',
'VIPCLOTHNG.NS',
'VIPIND.NS',
'VIPULLTD.NS',
'VISAKAIND.NS',
'VISASTEEL.NS',
'VISHNU.NS',
'VIVIDHA.NS',
'VIVIMEDLAB.NS',
'VLSFINANCE.NS',
'VMART.NS',
'VOLTAMP.NS',
'VOLTAS.NS',
'VRLLOG.NS',
'VSCL.NS',
'VSSL.NS',
'VSTIND.NS',
'VSTTILLERS.NS',
'VTL.NS',
'WABAG.NS',
'WABCOINDIA.NS',
'WALCHANNAG.NS',
'WANBURY.NS',
'WEBELSOLAR.NS',
'WEIZFOREX.NS',
'WEIZMANIND.NS',
'WELCORP.NS',
'WELENT.NS',
'WELINV.NS',
'WELSPUNIND.NS',
'WENDT.NS',
'WHEELS.NS',
'WHIRLPOOL.NS',
'WILLAMAGOR.NS',
'WINDMACHIN.NS',
'WINSOME.NS',
'WIPL.NS',
'WIPRO.NS',
'WOCKPHARMA.NS',
'WONDERLA.NS',
'WORTH.NS',
'WSI.NS',
'WSTCSTPAPR.NS',
'XCHANGING.NS',
'XLENERGY.NS',
'XPROINDIA.NS',
'YESBANK.NS',
'ZANDUREALT.NS',
'ZEEL.NS',
'ZEELEARN.NS',
'ZEEMEDIA.NS',
'ZENITHEXPO.NS',
'ZENSARTECH.NS',
'ZENTEC.NS',
'ZICOM.NS',
'ZODIACLOTH.NS',
'ZODJRDMKJ.NS',
'ZOTA.NS',
'ZUARI.NS',
'ZUARIGLOB.NS',
'ZYDUSWELL.NS',
'ZYLOG.NS']

stocks = [
'YESBANK.NS',
'NIFTYBEES.NS',
'RELIANCE.NS',
#'LTI.NS',
'MRF.NS',
'INFY.NS',
#'MOLDTKPAC.NS',
'CYIENT.NS',
'HDFCBANK.NS',
'PNB.NS',
'SOUTHBANK.NS',
'PERSISTENT.NS',
'DHFL.NS',
'TALWALKARS.NS',
'SANGHVIMOV.NS',
#'SKMEGGPROD-EQ.NS',
'GLAXO.NS',
'DIVISLAB.NS',
'GSKCONS.NS',
#'MPSLTD-EQ.NS',
'SONATSOFTW.NS',
'TCS.NS',
'MAYURUNIQ.NS',
'NAVNETEDUL.NS',
'COALINDIA.NS',
'GOLDBEES.NS',
#'BYKE.NS',
'TORNTPHARM.NS',
'SUZLON.NS',
'MOIL.NS',
'TATAMOTORS.NS',
'IDFC.NS',
#'IDFCBANK.NS',
'FORTIS.NS',
'OPTOCIRCUI.NS']

#end_date = '2017-11-23'

In [101]:
def normalize_data(train_data,test_data):
    from sklearn.preprocessing import normalize

    #print (train_data.iloc[1])
    train_data = normalize(train_data,axis=0)
    #train_target = normalize(train_target)
    test_data = normalize(test_data,axis=0)
    #test_target = normalize(test_target)
    #return train_data, test_data
    
    #print (train_data[1])
    train_data = normalize(train_data,axis=1)
    #train_target = normalize(train_target)
    test_data = normalize(test_data,axis=1)
    #test_target = normalize(test_target)
   
    #print (train_data[1])
    
    return train_data, test_data

In [102]:
def run_predictions_for_t_column(model_performance,t_signal,first_run,features,train_data,train_target,test_data,test_target):
    from sklearn.metrics import accuracy_score
    import operator
    from sklearn.metrics import confusion_matrix
    import sklearn as sk
    from sklearn.linear_model import LogisticRegression
    features_columns = features.columns
    baseline = len(test_target[test_target == 0])/len(test_target)
    
    for penalty in ('l1','l2'):
        #for maxiter in (1000,2000,3000,4000,5000,10000,50000,100000):
        for maxiter in [4000]:
            #for tolerance in (1e+10,1e+8,1e+6,1e+4,1e+2,1e+0,1e-2,1e-4,1e-6,1e-8,1e-10,1e-12):
            for tolerance in [1e-6]:
                #print ('processing '  +str(t_signal))
                model = sk.linear_model.LogisticRegression(penalty=penalty,max_iter=maxiter,tol=tolerance)
                #model.fit(train_data.as_matrix(),train_target.as_matrix())
                model.fit(train_data,train_target)
                #model.coef_
                #features_columns
                map_coeff_columns = dict()
                for i in range(len(features_columns)):
                    map_coeff_columns[features_columns[i]] = model.coef_[0][i]
                #print (map_coeff_columns)
                sorted_map_coeff_columns = sorted(map_coeff_columns.items(),key=operator.itemgetter(1),reverse=True)
                #sorted_map_coeff_columns
                #accuracy = accuracy_score(y_true=test_target.as_matrix(), y_pred=model.predict(test_data))
                accuracy = accuracy_score(y_true=test_target, y_pred=model.predict(test_data))
                #print ("Test Accuracy: %s" % accuracy)
                #print ("Baseline accuracy (majority class classifier): %s" % baseline)
                predicted_signal = model.predict(test_data)
                cmat = confusion_matrix(y_true=test_target.as_matrix(),
                                        y_pred=model.predict(test_data),
                                        labels=model.classes_)    # use the same order of class as the LR model.
                #print (' target_label | predicted_label | count ')
                #print ('--------------+-----------------+-------')
                # Print out the confusion matrix.
                # NOTE: Your tool may arrange entries in a different order. Consult appropriate manuals.
                false_action = 0
                false_no_action = 0
                true_action = 0
                true_no_action = 0
                for i, target_label in enumerate(model.classes_):
                    for j, predicted_label in enumerate(model.classes_):
                        #print ('{0:^13} | {1:^15} | {2:5d}'.format(target_label, predicted_label, cmat[i,j]))
                        if target_label == predicted_label:
                            if target_label ==0:
                                true_no_action = cmat[i,j]
                            else:
                                true_action = cmat[i,j]
                        elif target_label == 0:
                            false_action = cmat[i,j]
                        else:
                            false_no_action = cmat[i,j]
                        if (true_action + false_action) == 0:
                            precision = 0
                        else:
                            precision = true_action / (true_action + false_action)
                        if (true_action + false_no_action) == 0:
                            recall = 0
                        else:
                            recall = true_action/ (true_action + false_no_action)
                if accuracy > baseline:
                    if precision > 0.8:
                        predict = True
                    else:
                        predict = False
                else:
                    predict = False                    

                # save the model to disk
                model_name = stock + '_2017_11_23' +str(t_signal) +str('_') +str(penalty) +str('_') + str (maxiter) +str('_') + str(tolerance) +str('.sav')
                target_file_path = target_folder + model_name
                pickle.dump(model, open(target_file_path, 'wb'))
                new_row = {'Stock':stock,'Date':['2017-11-21'],'Prediction':t_signal,'Penalty':penalty,'Max_Iter':maxiter,
                           'Tolerance':tolerance,'Model_Coeff': str(sorted_map_coeff_columns),'Accuracy':accuracy,
                           'Majority_Class':baseline,'Confusion_Matrix':str(cmat),'Model_Classes':str(model.classes_),
                           'CM_True_No_Action':true_no_action,'CM_True_Buy/Sell':true_action,'CM_False_Buy/Sell':false_action,
                           'CM_False_Inaction':false_no_action,'Precision':precision,'Recall':recall,'Predict':predict}

                model_performance = model_performance.append(new_row,ignore_index=True)

                #model_name ='Yesbank_model_OneVsRest_2017-10-26.sav'
                #target_file_path = target_folder + model_name
                #pickle.dump(model_OneVsRest, open(target_file_path, 'wb'))
    return model_performance

In [103]:
#file_name = 'Consolidated_data_set.csv'
first_run = True
new_row = {'Stock':'dummy','Date':['2017-11-23'],'Prediction':[0],'Penalty':[0],'Max_Iter':[0],'Tolerance':[0],
           'Model_Coeff': [0],'Accuracy':[0],'Majority_Class':[0],'Confusion_Matrix':[0],'Model_Classes':[0],
           'CM_True_No_Action':[0],'CM_True_Buy/Sell':[0],'CM_False_Buy/Sell':[0],'CM_False_Inaction':[0],'Precision':[0],
           'Recall':[0],'Predict':[' ']}
model_performance = pd.DataFrame(data=new_row,index=None,columns=('Stock','Date','Prediction', 'Penalty','Max_Iter',
                                                                  'Tolerance','Model_Coeff','Accuracy','Majority_Class',
                                                                  'Confusion_Matrix','CM_True_No_Action','CM_True_Buy/Sell',
                                                                  'CM_False_Buy/Sell','CM_False_Inaction','Precision',
                                                                  'Recall','Predict'))

for stock in stocks:
    file_name = str(stock) + str(end_date) +str('.csv')
    #print (file_name)
    #model_name ='Comprehensive_model.sav'
    source_file_path = source_folder +  file_name
    #target_file_path = target_folder + model_name
    try:
        stock_data = pd.read_csv(source_file_path)
        skip = False
    except:
        print ('skipping ' +str(stock))
        skip = True
    
    if skip == False:
        stock_data = stock_data.replace([np.inf, -np.inf], np.nan)
        stock_data.fillna(0,inplace=True)
        attr_cols = (stock_data.columns).tolist()
        feature_cols = []
        signal_cols = []
        for i in attr_cols:
            attr_list=[]
            attr_list = i.split('_')
            if 'RBC' in attr_list or 'DBC' in attr_list or 'RBCV' in attr_list or 'DBCV' in attr_list  or 'Add' in attr_list:
                feature_cols.append(i)
            if 'Signal' in attr_list and 'V' not in attr_list:
                signal_cols.append(i)
        features = stock_data[feature_cols]
        features = features[200:]
        for t_signal in signal_cols:
            #print (t_signal)
            target = stock_data[t_signal]
            target = target[200:]
            train_data, test_data, train_target, test_target = train_test_split(features, target, test_size=0.25, random_state=42)
            if train_target[train_target == 1].count() > 0 and test_target[test_target == 1].count() > 0 and train_data.shape[0] > 1000:
                train_data, test_data = normalize_data(train_data,test_data)
                model_performance = run_predictions_for_t_column(model_performance,t_signal,first_run,features,train_data,train_target,test_data,test_target)
                first_run = False

skipping AARVI.NS
skipping ACCORD.NS
skipping ACEINTEG.NS
skipping AGROPHOS.NS
skipping AIRAN.NS
skipping AIROLAM.NS
skipping AISL.NS
skipping ARROWGREEN.NS
skipping ATUL.NS
skipping AURDIS.NS
skipping BANSAL.NS
skipping BETA.NS
skipping BHAGYANGR.NS
skipping BOHRA.NS
skipping CADSYS.NS
skipping CAPLIPOINT.NS
skipping CMMIPL.NS
skipping CREATIVE.NS
skipping DEVIT.NS
skipping DPABHUSHAN.NS
skipping DPWIRES.NS
skipping ECLFINANCE.NS
skipping EHFLNCD.NS
skipping EIFFL.NS
skipping EMKAYTOOLS.NS
skipping EROSMEDIA.NS
skipping FOCUS.NS
skipping FORGE.NS
skipping FOURTHDIM.NS
skipping GEEKAYWIRE.NS
skipping GLOBAL.NS
skipping GOLDSTAR.NS
skipping GRUH.NS
skipping HECPROJECT.NS
skipping HITECH.NS
skipping HMT.NS
skipping IIFCL.NS
skipping IIFLFIN.NS
skipping IIHFL.NS
skipping INDIGRID.NS
skipping INDSWFTLAB.NS
skipping INFOBEAN.NS
skipping INNOVATIVE.NS
skipping IRBINVIT.NS
skipping IREDA.NS
skipping IRFC.NS
skipping ISMTLTD.NS
skipping IVC.NS
skipping IVP.NS
skipping JALAN.NS
skipping JASH.NS

In [105]:
file_name = str('Model_Performance_') + str(end_date) +str('.csv')
target_file_path = target_folder + file_name
model_performance.to_csv(target_file_path)

In [63]:
#precision = model.evaluate(test_data, metric='precision')['precision']
#print "Precision on test data: %s" % precision

#from sklearn.metrics import precision_score
#precision = precision_score(y_true=test_target.as_matrix(), 
#                            y_pred=model.predict(test_data))
#print ("Precision on test data: %s" % precision)

In [64]:
#recall = model.evaluate(test_data, metric='recall')['recall']
#print "Recall on test data: %s" % recall

#from sklearn.metrics import recall_score
#recall = recall_score(y_true=test_target.as_matrix(),
#                      y_pred=model.predict(test_data))
#print ("Recall on test data: %s" % recall)

In [65]:
#model.predict_log_proba(train_data)

In [66]:
#model.predict_proba(test_data)

In [67]:
#def apply_threshold(probabilities, threshold, predicted_signal):
    ### YOUR CODE GOES HERE
    # +1 if >= threshold and -1 otherwise.
#    prediction = pd.Series(len(probabilities))
#    for i in range(len(probabilities)):
#        if probabilities[i] > threshold:
#            prediction[i] = predicted_signal[i]
#        #elif predicted_signal[i] == 0:
        #    prediction[i] = 0
        #elif predicted_signal[i] == -1:
        #    prediction[i] = -1
#        else:
#            prediction[i] = 2 # unless strong signal, take no position (buy/sell)
#    return prediction

In [68]:
#probabilities = model.predict_proba(test_data)[:,1]
#predictions_with_default_threshold = apply_threshold(probabilities, 0.5, predicted_signal)
#predictions_with_high_threshold = apply_threshold(probabilities, 0.8, predicted_signal)

In [69]:
#from sklearn.metrics import confusion_matrix
#cmat = confusion_matrix(y_true=test_target.as_matrix(),
#                        y_pred=predictions_with_high_threshold,
#                        labels=model.classes_)    # use the same order of class as the LR model.
#print (' target_label | predicted_label | count ')
#print ('--------------+-----------------+-------')
# Print out the confusion matrix.
# NOTE: Your tool may arrange entries in a different order. Consult appropriate manuals.
#for i, target_label in enumerate(model.classes_):
#    for j, predicted_label in enumerate(model.classes_):
#        print ('{0:^13} | {1:^15} | {2:5d}'.format(target_label, predicted_label, cmat[i,j]))

In [70]:
#type(probabilities)

In [71]:
#prob_df = pd.DataFrame(probabilities)

In [72]:
#prob_df[0].hist()
#plt.show()

In [73]:
#prob_df.describe()

In [74]:
#from sklearn.preprocessing import label_binarize
#from sklearn import svm, datasets

# Use label_binarize to be multi-label like settings
#Y = label_binarize(target, classes=[-1, 0, 1])
#n_classes = Y.shape[1]
#X = features

# Split into training and test
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25, random_state=42)

# We use OneVsRestClassifier for multi-label prediction
#from sklearn.multiclass import OneVsRestClassifier

# Run classifier
#classifier = OneVsRestClassifier(svm.LinearSVC(random_state=42))
#model_OneVsRest = classifier.fit(X_train, Y_train)
#y_score = classifier.decision_function(X_test)
#y_pred = classifier.predict(X_test)

In [75]:
#y_score

In [76]:
#y_pred

In [77]:
#from sklearn.metrics import average_precision_score
#from sklearn.metrics import average_recall_score
#average_precision = average_precision_score(Y_test, y_score)
#average_recall = average_recall_score(Y_test,y_score)

#print('Average precision-recall score: {0:0.2f}'.format(average_precision))

In [78]:
#from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import average_precision_score

# For each class
#precision = dict()
#recall = dict()
#average_precision = dict()
#for i in range(n_classes):
#    precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
#                                                        y_score[:, i])
#    average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
#precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
#    y_score.ravel())
#average_precision["micro"] = average_precision_score(Y_test, y_score,
#                                                     average="micro")
#print('Average precision score, micro-averaged over all classes: {0:0.2f}'

#      .format(average_precision["micro"]))

In [79]:
#y_score

In [80]:
#Y_test.shape

In [81]:
#Y_test

In [82]:
#match = 0
#minus_1 = 0
#zero = 0
#plus_1 = 0
#unknown = 0
#for i in range(3):
#    for x in range(len(Y_test)):
#        if Y_test[x][i] == y_pred[x][i]:
#            match = match + 1
#        elif y_pred[x][0] == 1:
#            minus_1 = minus_1 + 1
#        elif y_pred[x][1] == 1:
#            zero = zero + 1
#        elif y_pred[x][2] == 1:
#            plus_1 = plus_1 + 1
#        else:
#            unknown = unknown + 1
#    print (str(i) +' matched ' +str(match))
#    print (str(minus_1) +' ' +str(zero) +' ' +str(plus_1) +' ' +str(unknown))
#    match = 0
#    minus_1 = 0
#    zero = 0
#    plus_1 = 0
#    unknown = 0
#print ('total rows ' +str(len(Y_test))) 

In [83]:
#model_read = pickle.load(open(target_file_path, 'rb'))

In [84]:
#model_read

In [85]:
#a = y_pred[0]

In [86]:
#no_pred = 0
#for i in range(len(y_pred)):
#    if y_pred[i].sum() > 1:
#        no_pred += 1
#        print ('no pred ' +str(y_pred[i]))
#print ('no pred ' +str(no_pred))